# GPT-4o with SAP AI Core - Example Notebook

This notebook demonstrates how to use GPT-4o through SAP AI Core's Gen AI Hub.

## Prerequisites

1. SAP AI Core service key configured
2. Gen AI Hub SDK installed: `pip install generative-ai-hub-sdk`
3. AI Core credentials configured using `aicore configure -k <path-to-service-key>`


## 1. Setup and Import Required Libraries


In [3]:
# Install required packages (run once)
!pip3 install generative-ai-hub-sdk requests python-dotenv



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [4]:
import os
import json
from gen_ai_hub.proxy.native.openai import OpenAI
from gen_ai_hub.proxy.core.proxy_clients import get_proxy_client


In [5]:
import pkg_resources 
print(pkg_resources.get_distribution('gen-ai-hub').version)

DistributionNotFound: The 'gen-ai-hub' distribution was not found and is required by the application

## 2. Configure AI Core Credentials

### Option A: Using Service Key File


In [24]:
# Path to your AI Core service key JSON file
service_key_path = "../aicore-service-key.json"

# Load service key
with open(service_key_path, 'r') as f:
    service_key = json.load(f)

# Set environment variables
os.environ['AICORE_AUTH_URL'] = service_key['url']
os.environ['AICORE_CLIENT_ID'] = service_key['clientid']
os.environ['AICORE_CLIENT_SECRET'] = service_key['clientsecret']
os.environ['AICORE_RESOURCE_GROUP'] = 'default'

# Parse serviceurls - it might be a string or already a dict
serviceurls = service_key['serviceurls']
if isinstance(serviceurls, str):
    serviceurls = json.loads(serviceurls)
os.environ['AICORE_BASE_URL'] = serviceurls['AI_API_URL']

print("✓ AI Core credentials configured")


✓ AI Core credentials configured


### Option B: Using CLI Configuration

If you've already configured using `aicore configure`, you can skip Option A:


In [25]:
# If already configured via CLI, credentials are stored in ~/.aicore/config.json
# No additional setup needed - SDK will automatically use stored credentials
print("✓ Using existing AI Core configuration")


✓ Using existing AI Core configuration


## 3. Initialize OpenAI Client


In [26]:
# Initialize the OpenAI client through SAP AI Core proxy
client = OpenAI()

print("✓ OpenAI client initialized")


✓ OpenAI client initialized


## 4. Basic Chat Completion with GPT-4o


In [1]:
# Simple chat completion example
try:
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a helpful AI assistant."},
            {"role": "user", "content": "What is SAP AI Core?"}
        ],
        max_tokens=500,
        temperature=0.7
    )
    
    # Check if response has choices
    if response.choices and len(response.choices) > 0:
        print("Response:")
        print(response.choices[0].message.content)
        print(f"\nTokens used: {response.usage.total_tokens}")
    else:
        print("Error: No response choices returned from the API")
        print(f"Full response: {response}")
        
except Exception as e:
    print(f"Error calling GPT-4o: {e}")
    print(f"Error type: {type(e).__name__}")


Error calling GPT-4o: name 'client' is not defined
Error type: NameError


## 5. Streaming Response


In [18]:
# Streaming example for real-time response
print("Streaming response:\n")

try:
    stream = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": "Write a short poem about AI and cloud computing."}
        ],
        stream=True,
        max_tokens=200
    )
    
    for chunk in stream:
        if chunk.choices and len(chunk.choices) > 0:
            if chunk.choices[0].delta.content is not None:
                print(chunk.choices[0].delta.content, end="")
    
    print("\n\n✓ Streaming complete")
    
except Exception as e:
    print(f"\n\nError during streaming: {e}")
    print(f"Error type: {type(e).__name__}")


Streaming response:

In the vast domain where data flows,  
Clouds weave the threads that knowledge sews,  
AI's whispers, soft and clear,  
Guide us through, or lend an ear.  

Electric thoughts in circuits race,  
Learning patterns, keeping pace.  
Within the cloud, a cosmic dance,  
Of ones and zeros, a chance romance.  

Together they sculpt a virtual sky,  
Where dreams and reality unify.  
Boundless realms of code's embrace,  
In synergy, they carve out space.  

From human minds, a spark ignites,  
With cloud and AI in shared flights,  
Towards futures bright with fathomless scope,  
In the ether's art, we find our hope.  

✓ Streaming complete


## 6. Multi-turn Conversation


In [19]:
# Maintain conversation history
conversation_history = [
    {"role": "system", "content": "You are a helpful SAP technology expert."},
    {"role": "user", "content": "What is SAP BTP?"}
]

try:
    # First exchange
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=conversation_history,
        max_tokens=300
    )
    
    if response.choices and len(response.choices) > 0:
        assistant_message = response.choices[0].message.content
        conversation_history.append({"role": "assistant", "content": assistant_message})
        
        print("Assistant:", assistant_message)
        print("\n" + "="*80 + "\n")
        
        # Follow-up question
        conversation_history.append({"role": "user", "content": "How does it integrate with AI Core?"})
        
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=conversation_history,
            max_tokens=300
        )
        
        if response.choices and len(response.choices) > 0:
            print("User: How does it integrate with AI Core?")
            print("\nAssistant:", response.choices[0].message.content)
        else:
            print("Error: No response for follow-up question")
    else:
        print("Error: No response for initial question")
        
except Exception as e:
    print(f"Error in conversation: {e}")
    print(f"Error type: {type(e).__name__}")


Assistant: SAP BTP stands for SAP Business Technology Platform. It is a comprehensive platform that provides a suite of services and solutions for enterprises to accelerate their digital transformation. SAP BTP integrates various technologies and services, enabling organizations to develop, deploy, and manage applications and services effectively. Here are its key components:

1. **Database and Data Management**: SAP BTP offers powerful database solutions including SAP HANA, which provides advanced data processing capabilities. It also includes data integration and data warehousing solutions to handle large volumes of data efficiently.

2. **Analytics**: The platform offers tools for business intelligence, planning, and predictive analytics. SAP Analytics Cloud is part of this layer, enabling organizations to gain insights from their data through intuitive dashboards and reporting.

3. **Application Development and Integration**: SAP BTP provides tools for developing, deploying, and ex

## 7. Advanced Configuration Options


In [20]:
# Example with various parameters
try:
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a creative writer."},
            {"role": "user", "content": "Write a creative product description for an AI-powered chatbot."}
        ],
        max_tokens=300,
        temperature=0.9,        # Higher = more creative (0.0 - 2.0)
        top_p=0.95,            # Nucleus sampling
        frequency_penalty=0.5,  # Reduce repetition (-2.0 - 2.0)
        presence_penalty=0.3,   # Encourage new topics (-2.0 - 2.0)
        n=1                     # Number of completions to generate
    )
    
    if response.choices and len(response.choices) > 0:
        print("Creative Description:")
        print(response.choices[0].message.content)
        print(f"\nFinish reason: {response.choices[0].finish_reason}")
        print(f"Model: {response.model}")
    else:
        print("Error: No response choices returned")
        
except Exception as e:
    print(f"Error with advanced configuration: {e}")
    print(f"Error type: {type(e).__name__}")


Creative Description:
**Introducing ChatMaster 3000: Your Ultimate AI Conversational Companion**

Step into the future of communication with ChatMaster 3000, the most advanced AI-powered chatbot designed to transform every interaction into a delightful experience. Crafted with cutting-edge technology and an intuitive design, ChatMaster 3000 is not just a tool—it's your gateway to seamless conversations and intelligent interactions.

**Features:**

- **Intelligent Conversations:** Equipped with natural language processing capabilities, ChatMaster 3000 understands context and nuances like never before. Whether discussing complex topics or casual chit-chat, it adapts to your style.

- **24/7 Availability:** Never wait for a response again! ChatMaster 3000 is always on, ready to assist with any query at any time. Whether you’re planning your day or seeking support, it’s there when you need it.

- **Personalized Interactions:** Learning from every conversation, ChatMaster 3000 evolves along

## 8. Error Handling and Best Practices


In [21]:
from openai import OpenAIError, RateLimitError, APIError
import time

def safe_chat_completion(messages, max_retries=3, retry_delay=2):
    """Wrapper with error handling and retry logic"""
    
    for attempt in range(max_retries):
        try:
            response = client.chat.completions.create(
                model="gpt-4o",
                messages=messages,
                max_tokens=500,
                temperature=0.7
            )
            return response
            
        except RateLimitError as e:
            print(f"Rate limit exceeded. Retrying in {retry_delay} seconds...")
            time.sleep(retry_delay)
            retry_delay *= 2  # Exponential backoff
            
        except APIError as e:
            print(f"API Error: {e}")
            if attempt < max_retries - 1:
                time.sleep(retry_delay)
            else:
                raise
                
        except OpenAIError as e:
            print(f"OpenAI Error: {e}")
            raise
            
    raise Exception("Max retries exceeded")

# Use the wrapper
try:
    response = safe_chat_completion([
        {"role": "user", "content": "Explain SAP AI Core in one sentence."}
    ])
    print(response.choices[0].message.content)
except Exception as e:
    print(f"Failed to get response: {e}")


SAP AI Core is a scalable, high-performance AI foundation service that enables the management and deployment of AI models and workflows within SAP's ecosystem.


## Summary

This notebook covered:
- ✅ Setting up SAP AI Core credentials
- ✅ Basic chat completions
- ✅ Streaming responses
- ✅ Multi-turn conversations
- ✅ Advanced configuration options
- ✅ Error handling and retries

## Additional Resources

- [SAP AI Core Documentation](https://help.sap.com/docs/AI_CORE)
- [Gen AI Hub SDK Documentation](https://github.com/SAP/generative-ai-hub-sdk)
- [OpenAI API Reference](https://platform.openai.com/docs/api-reference)
